**HELPER FUNCTIONS**

In [3]:
import json, re

def getHTSRecord(path: str) -> list[dict[str, any]]:


    with open(path, 'r') as file:
        return json.loads(file.read())
    
def checkFinalQueryState(finalRecord: str, queryState: str) -> bool:

    if finalRecord == queryState:
        return True
    else:
        return False

**MAIN FUNCTIONS**

In [43]:
def queryRecord(query: list, htsRecord: list[dict[str, any]]):

    queryObj = {
        'query': query,
        'followup': [],
        'records': [],
        'finalRecord': query[len(query)-1]
    }
    indent = 0
    fIndex = 0
    currentIndex = 0
    run = True

    while run:

        for index, record in enumerate(htsRecord):
            
            if index == (len(htsRecord) -1):
                print('End of htsRecord')
                run = False
                break
            else:
                print(len(htsRecord), index)
                queryObj['followup'].append(htsRecord[index+1])
                fIndex += 1    
            
            queryState = queryObj['query'][currentIndex]

            expression = re.compile(rf"{queryState}")

            #Case where the current record iterated matches the current queryState (even if it is not a full match)
            if 'htsno' in record and re.match(expression, record['htsno']):

                #Check if we are at the end of the query
                if checkFinalQueryState(queryObj['finalRecord'], queryState):

                    print(f"Found final record with description: {record['description']}")
                    run = False
                    break

                ##Check for the queryObj.query and add necessary changes to the queryObj
                print(f'Found hts record with description: {record["description"]}')
                indent += 1
                currentIndex += 1

            #Case where the following record to an EH has an htsno key
            elif (record['indent'] == indent) and ('htsno' in queryObj['followup'][fIndex-1]):

                print(f'Found one EH with description: {record["description"]}')
                if re.match(expression, queryObj['followup'][fIndex-1]['htsno']):
                    print(f"Found following htsno after EH: {queryObj['followup'][fIndex-1]['description']}")





In [45]:
query = ["0101", "0101.21", "0101.21.00", "0101.21.00.10"]
htsRecord = getHTSRecord('../../db_hts/temp/NEW_final_json_files/0101.json')

queryRecord(query, htsRecord)    

12 0
Found hts record with description: Live horses, asses, mules and hinnies:
12 1
Found one EH with description: Horses:
Found following htsno after EH: Purebred breeding animals
12 2
Found hts record with description: Purebred breeding animals
12 3
Found hts record with description: Males
12 4
Found one EH with description: Females
12 5
12 6
Found one EH with description: Imported for immediate slaughter
12 7
Found one EH with description: Other
12 8
12 9
12 10
End of htsRecord
